In [33]:
from main import run_main
import json
import pandas as pd
import time

In [34]:
CROSSOVER_METHODS = ["one_point", "double_point", "uniform", "anular"]
SELECTION_METHODS = ["elite", "roulette", "universal", "boltzmann", "deterministic_tournament", "probabilistic_tournament", "ranking"]
MUTATION_METHODS = ["gene_mutation","uniform_multigen", "complete_mutation", "limited_multigen"]
GENES = ["items", "class", "height"]
NEW_GENERATION_METHODS = ["use_all", "new_over_actual"]
CHARACTER_TYPES = ["warrior", "archer", "defender", "infiltrator"]

In [35]:
with open('./config.json', 'r') as f:
        config = json.load(f)

In [36]:


totalIterations = (len(CROSSOVER_METHODS) * len(SELECTION_METHODS) * 
                   len(MUTATION_METHODS) * len(GENES) * len(NEW_GENERATION_METHODS) * 
                   len(CHARACTER_TYPES) * 5)

currentIteration = 0
startTime = time.time()

df = []

for cross in CROSSOVER_METHODS:
    for sel1 in SELECTION_METHODS:
        for mut in MUTATION_METHODS:
            for gen in GENES:
                for new_gen in NEW_GENERATION_METHODS:
                    for clase in CHARACTER_TYPES:
                        for _ in range(10):
                            config["character_type"] = clase
                            config["crossing_method"] = cross
                            config["gene"] = gen
                            config["mutation_method"] = mut
                            config["new_generation_method"] = new_gen
                            config["selection_method_1"] = sel1
                            config["selection_method_2"] = sel1
                            config["selection_method_3"] = sel1
                            config["selection_method_4"] = sel1
                            try:
                                df.append((run_main(config), config))
                            except OverflowError:
                                print(config)

                            currentIteration += 1
                            
                            if currentIteration % 100 == 0:  # Actualizar cada 100 iteraciones
                                elapsedTime = time.time() - startTime
                                estimatedRemainingTime = (totalIterations - currentIteration) * (elapsedTime / currentIteration)
                                percentageCompletion = (currentIteration / totalIterations) * 100
                                print(f"\rCompletion: {percentageCompletion:.2f}% ETA: {estimatedRemainingTime:.2f}s", end="")

Completion: 58.78% ETA: 127.26s

KeyboardInterrupt: 

In [ ]:
totalIterations = (len(CROSSOVER_METHODS) * len(SELECTION_METHODS) * 
                   len(MUTATION_METHODS) * len(GENES) * len(NEW_GENERATION_METHODS) * 
                   len(CHARACTER_TYPES) * 10)
totalIterations

26880

In [ ]:
df2 = [{**{"strength": char[0]._items["strength"],
           "agility": char[0]._items["agility"],
           "expertise": char[0]._items["expertise"],
           "endurance": char[0]._items["endurance"],
           "health": char[0]._items["health"],
           "height": char[0]._height,
           "performance": char[0].get_performance(),
           "generation": char[1]},
        **config}
       for char, config in df]

df2 = pd.DataFrame(df2)
df2